<a href="https://colab.research.google.com/github/componavt/topkar-space/blob/main/src/ner/stanza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install stanza
!pip install pandas
!pip install nltk


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------- ----- 1.3/1.5 MB 9.7 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 6.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import stanza

# Загружаем модель для русского языка
stanza.download('ru')  # Загрузите модель, если она еще не загружена
nlp = stanza.Pipeline('ru')

# Пример текста на русском языке
text = "Дом в дер. Койвусельга, фам. Михайлов. ="

# Обрабатываем текст
doc = nlp(text)

# Извлекаем именованные сущности
for sentence in doc.sentences:
    for entity in sentence.ents:
        print(f'Текст: {entity.text}, Тип: {entity.type}')


c:\Users\Ярослав\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-22 04:46:01 INFO: Downloaded file to C:\Users\Ярослав\stanza_resources\resources.json
2025-09-22 04:46:01 INFO: Downloading default packages for language: ru (Russian) ...
2025-09-22 04:47:03 INFO: Downloaded file to C:\Users\Ярослав\stanza_resources\ru\default.zip
2025-09-22 04:47:07 INFO: Finished downloading models and saved to C:\Users\Ярослав\stanza_resources
2025-09-22 04:47:07 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-09-22 04:47:08 INFO: Downloaded file to C:\Users\Ярослав\stanza_resources\resources.json
2025-09-22 04:47:10 INFO: Loading the

Текст: Михайлов, Тип: LOC


In [6]:
import pandas as pd

csv_files = [
#    "https://raw.githubusercontent.com/componavt/topkar-space/main/data/sample10.csv",
    "https://raw.githubusercontent.com/componavt/topkar-space/main/data/sample100.csv",
]

df = pd.concat([pd.read_csv(url, sep = ';') for url in csv_files], ignore_index=True)
df = df.reset_index()  # make sure indexes pair with number of rows
df.head()

,index,Name,Synonym,Text
0,0,Наволоцкая поляна,наволоцкая поляна,Пахотная поляна в устье Пижейручья.
1,1,Галайские пожни,галайские пожни,"покосы на юге острова Галайский, см. Галайский..."
2,2,Pahag’ärv,pahagärv,"Маленькое озерко за хутором Poh'd'ad'g', между..."
3,3,Sod’järvt’e,sodjärvte,Поляны по дороге в дер. Сидорово.
4,4,Piirdoinkodi,piirdoinkodi,"Дом в дер. Койвусельга, фам. Михайлов."


In [7]:
import stanza

text = "Яблоко рассматривает возможность покупки стартапа в Великобритании за 1 миллиард долларов. Владимир Путин является президентом России. А ещё паста и Кижи"

doc = nlp(text)

for sentence in doc.sentences:
    for entity in sentence.ents:
        print(f'Текст: {entity.text}, Тип: {entity.type}')


Текст: Великобритании, Тип: LOC
Текст: Владимир Путин, Тип: PER
Текст: России, Тип: LOC
Текст: Кижи, Тип: LOC


In [14]:
from asyncio.windows_events import NULL
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
lines = df['Text'].tolist()
print()
print("Number of toponyms:", len(lines))
toponyms = []
df['Text'] = df['Text'].replace({float('nan'): ""})
num=0
output_csv = 'output.csv'
with open(output_csv, 'w', encoding='utf-8') as outfile:
  outfile.writelines(f"sentence_id; toponyms_list \n")
  if sentence != NULL:
    for sentence in lines:
        tex = nlp(sentence)
        print(f'\nSentence {num}: {sentence}')
        num+=1
        for entity in tex.ents:
          if entity.type == "LOC":
            print(f'Location: {entity.text}')
            # print(f'Location: {entity.text}, type: {entity.type}')
            toponyms.append(entity.text)
        strk = (f"{num}; {toponyms} \n")
        strk = strk.replace('[', '').replace(']', '').replace('\'', '').replace('\"', '')
        outfile.writelines(strk)
        toponyms = []

#print("Число найденных топонимов", len(toponyms))


Number of toponyms: 100

Sentence 0: Пахотная поляна в устье Пижейручья.
Location: Пижейручья

Sentence 1: покосы на юге острова Галайский, см. Галайский Остров
Location: Галайский
Location: Галайский Остров

Sentence 2: Маленькое озерко за хутором Poh'd'ad'g', между оз. Sar'gär'v и Vougedg'är'v.
Location: Poh'd'ad'g'
Location: Sar'gär'v
Location: Vougedg'är'v

Sentence 3: Поляны по дороге в дер. Сидорово.
Location: Поляны
Location: Сидорово

Sentence 4: Дом в дер. Койвусельга, фам. Михайлов.
Location: Михайлов

Sentence 5: Река Колпь, вытекает из оз. Jokšar'v, течет в Вологодск. обл.
Location: Колпь
Location: Jokšar'v
Location: Вологодск

Sentence 6: Поле находится в 600-700 м на востоке д.Каскосельга.
Location: Каскосельга

Sentence 7: 

Sentence 8: Урочище, в котором , по сведениям информатора, в 30-х годах упала сосна. Тут болото.

Sentence 9: Покос у мостков через болото ( дер. Slabad).
Location: Slabad

Sentence 10: Гора у оз. Čern'eckiig'är'f, за дер. Селище.
Location: Селище



In [11]:
import csv

def replace_tabs_in_csv(input_file, output_file):
    with open(input_file, 'r', newline='', encoding='utf-8') as infile:

        content = infile.read()


    content = content.replace('	', '@')
    content = content.replace('\n', ' ')

    with open(output_file, 'w', newline='', encoding='utf-8') as outfile:

        outfile.write(content)


def remove_single_character_lines(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile:

        lines = infile.readlines()


    filtered_lines = [line for line in lines if len(line.strip()) != 1]

    with open(output_file, 'w', encoding='utf-8') as outfile:

        outfile.writelines(filtered_lines)


input_csv = 'topnames.csv'
output_csv = 'output.csv'
replace_tabs_in_csv(input_csv, output_csv)

input_txt = 'output.csv'
output_txt = 'topnames_csv.csv'
remove_single_character_lines(input_txt, output_txt)

print(f"Строки с одним символом в файле '{input_txt}' удалены и сохранены в '{output_txt}'.")



FileNotFoundError: [Errno 2] No such file or directory: 'topnames.csv'

In [22]:
"""
Comparison Analysis between output.csv and reference_table_sample100.csv
"""

import pandas as pd
import re
from collections import Counter

def clean_toponym_list(toponym_str):
    """Clean and normalize toponym lists for comparison"""
    if pd.isna(toponym_str) or toponym_str == '':
        return []
    
    # Split by comma and clean each toponym
    toponyms = [t.strip() for t in str(toponym_str).split(',')]
    # Remove empty strings
    toponyms = [t for t in toponyms if t]
    return toponyms

def compare_toponym_lists(list1, list2):
    """Compare two lists of toponyms and return differences"""
    set1 = set(list1)
    set2 = set(list2)
    
    only_in_1 = set1 - set2
    only_in_2 = set2 - set1
    common = set1 & set2
    
    return {
        'only_in_output': list(only_in_1),
        'only_in_reference': list(only_in_2),
        'common': list(common),
        'output_count': len(list1),
        'reference_count': len(list2),
        'common_count': len(common)
    }


# Read the datasets
print("Loading datasets...")

# Read output.csv
output_df = pd.read_csv('output.csv', sep=';')
print(f"Output dataset: {len(output_df)} rows")

# Read reference table
reference_df = pd.read_csv('https://raw.githubusercontent.com/componavt/topkar-space/main/data/reference_table_sample100.csv')
print(f"Reference dataset: {len(reference_df)} rows")

# Clean column names
output_df.columns = output_df.columns.str.strip()

print("\n" + "="*80)
print("DATASET STRUCTURE COMPARISON")
print("="*80)

print(f"\nOutput.csv columns: {list(output_df.columns)}")
print(f"Reference table columns: {list(reference_df.columns)}")

print("\n" + "="*80)
print("TOPONYM LIST COMPARISON")
print("="*80)

# Compare toponym lists
comparison_results = []

for idx in range(len(output_df)):
    output_row = output_df.iloc[idx]
    reference_row = reference_df.iloc[idx] if idx < len(reference_df) else None
    
    # Get toponyms from output.csv
    output_toponyms = clean_toponym_list(output_row['toponyms_list'])
    
    # Get toponyms from reference table
    if reference_row is not None:
        reference_toponyms = clean_toponym_list(reference_row['Toponim_list'])
    else:
        reference_toponyms = []
    
    # Compare the lists
    comparison = compare_toponym_lists(output_toponyms, reference_toponyms)
    comparison['sentence_id'] = output_row['sentence_id']
    comparison['output_text'] = output_row['toponyms_list']
    comparison['reference_text'] = reference_row['Toponim_list'] if reference_row is not None else ''
    
    comparison_results.append(comparison)

# Create summary statistics
total_only_in_output = sum(len(r['only_in_output']) for r in comparison_results)
total_only_in_reference = sum(len(r['only_in_reference']) for r in comparison_results)
total_common = sum(r['common_count'] for r in comparison_results)
total_output = sum(r['output_count'] for r in comparison_results)
total_reference = sum(r['reference_count'] for r in comparison_results)

print(f"\nSUMMARY STATISTICS:")
print(f"Total toponyms in output.csv: {total_output}")
print(f"Total toponyms in reference table: {total_reference}")
print(f"Common toponyms: {total_common}")
print(f"Only in output.csv: {total_only_in_output}")
print(f"Only in reference table: {total_only_in_reference}")

# Calculate accuracy metrics
if total_reference > 0:
    precision = total_common / total_output if total_output > 0 else 0
    recall = total_common / total_reference
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    print(f"\nACCURACY METRICS:")
    print(f"Precision: {precision:.3f} ({precision*100:.1f}%)")
    print(f"Recall: {recall:.3f} ({recall*100:.1f}%)")
    print(f"F1-Score: {f1_score:.3f} ({f1_score*100:.1f}%)")

# Find rows with significant differences
print(f"\n" + "="*80)
print("ROWS WITH SIGNIFICANT DIFFERENCES")
print("="*80)

significant_diffs = []
for r in comparison_results:
    if len(r['only_in_output']) > 0 or len(r['only_in_reference']) > 0:
        significant_diffs.append(r)

print(f"Found {len(significant_diffs)} rows with differences")

# Show first 10 significant differences
for i, diff in enumerate(significant_diffs[:10]):
    print(f"\nRow {diff['sentence_id']}:")
    print(f"  Output: '{diff['output_text']}'")
    print(f"  Reference: '{diff['reference_text']}'")
    if diff['only_in_output']:
        print(f"  Only in output: {diff['only_in_output']}")
    if diff['only_in_reference']:
        print(f"  Only in reference: {diff['only_in_reference']}")
    if diff['common']:
        print(f"  Common: {diff['common']}")

# Analyze most common differences
print(f"\n" + "="*80)
print("MOST COMMON DIFFERENCES")
print("="*80)

all_only_in_output = []
all_only_in_reference = []

for r in comparison_results:
    all_only_in_output.extend(r['only_in_output'])
    all_only_in_reference.extend(r['only_in_reference'])

print(f"\nMost frequent toponyms only in output.csv:")
output_counter = Counter(all_only_in_output)
for toponym, count in output_counter.most_common(10):
    print(f"  '{toponym}': {count} times")

print(f"\nMost frequent toponyms only in reference table:")
reference_counter = Counter(all_only_in_reference)
for toponym, count in reference_counter.most_common(10):
    print(f"  '{toponym}': {count} times")

# Save detailed comparison to CSV
comparison_df = pd.DataFrame(comparison_results)
comparison_df.to_csv('detailed_comparison.csv', index=False)
print(f"\nDetailed comparison saved to 'detailed_comparison.csv'")


Loading datasets...
Output dataset: 100 rows


HTTPError: HTTP Error 404: Not Found